In [1]:
# Exploring the adult data set from the UCI repository.
# Download the files adult.data and adult.test
# from https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
import matplotlib.pyplot as plt

In [2]:
from __future__ import print_function
import pandas as pd
import ipywidgets as widgets
import numpy as np
import adult
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from forest_clusters import ForestClusters, buckets, total_variation

original = adult.original
categorical = [col for col in original.columns 
               if ~np.issubdtype(original[col], np.number)]
original[categorical] = original[categorical].fillna(value = 'None')

In [3]:
X = pd.get_dummies(original[original.columns.difference(['Target'])])

def encode_features(original):
    return X.loc[original.index]

y = adult.labels

original_train, original_test, X_train, X_test, y_train, y_test = train_test_split(original, X, y, train_size=0.7)

forest = RandomForestClassifier(n_estimators = 30)
forest = forest.fit(X_train, y_train)
print("Score %f" % forest.score(X_test, y_test))

leaf_encoder = OneHotEncoder()
leaf_encoder = leaf_encoder.fit(forest.apply(X))

/home/arthur/env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Score 0.849519


In [4]:
n_clusters = 30
c = ForestClusters(forest, leaf_encoder, n_clusters = n_clusters)
c.fit(X_train.sample(n = 1000))

In [5]:
from display_cluster import display_cluster
display_cluster(c, original, original_test, y_test,
                encode_features=encode_features)

,Size,Accuracy,Mean Votes
0,10953,0.800055,8.457683
1,410,1.000000,0.039024
2,362,0.997238,29.198895
3,134,1.000000,0.149254
4,98,0.989796,0.377551
5,152,1.000000,0.401316
6,31,1.000000,0.709677
7,113,0.991150,0.168142
8,407,0.990172,0.152334
9,81,1.000000,0.234568


interactive(children=(BoundedIntText(value=0, description='Cluster', max=30), Dropdown(description='Column', o…